In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype='int')
dataset.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
indep=dataset[['Gender_Male','Age','EstimatedSalary']]
dep=dataset[['Purchased']]

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [7]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import GridSearchCV
param_grid={'alpha':[0.001,0.01,1.0], 'force_alpha':[True,'bool'], 'fit_prior':[True,'bool']}
grid=GridSearchCV(BernoulliNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
45 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 95, in validate_pa

GridSearchCV(estimator=BernoulliNB(), n_jobs=-1,
             param_grid={'alpha': [0.001, 0.01, 1.0],
                         'fit_prior': [True, 'bool'],
                         'force_alpha': [True, 'bool']},
             scoring='f1_weighted', verbose=3)

In [8]:
result=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [9]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print('confusion matrix:\n',cm)

confusion matrix:
 [[79  0]
 [41  0]]


In [10]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print('The clf report :\n',clf_report)

The clf report :
               precision    recall  f1-score   support

           0       0.66      1.00      0.79        79
           1       0.00      0.00      0.00        41

    accuracy                           0.66       120
   macro avg       0.33      0.50      0.40       120
weighted avg       0.43      0.66      0.52       120



C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print('The f1_macro value for the best parameters {}:'.format(grid.best_params_),f1_macro)

The f1_macro value for the best parameters {'alpha': 0.001, 'fit_prior': True, 'force_alpha': True}: 0.5226968174204355


In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.5146650200679223

In [13]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013225,0.002147,0.029812,0.002271,0.001,True,True,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,1
1,0.003226,0.004139,0.000000,0.000000,0.001,True,bool,"{'alpha': 0.001, 'fit_prior': True, 'force_alp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,0.003007,0.004008,0.000000,0.000000,0.001,bool,True,"{'alpha': 0.001, 'fit_prior': 'bool', 'force_a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
3,0.002026,0.002568,0.000000,0.000000,0.001,bool,bool,"{'alpha': 0.001, 'fit_prior': 'bool', 'force_a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,0.007333,0.006403,0.022532,0.008060,0.01,True,True,"{'alpha': 0.01, 'fit_prior': True, 'force_alph...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,1
5,0.002085,0.002062,0.000000,0.000000,0.01,True,bool,"{'alpha': 0.01, 'fit_prior': True, 'force_alph...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
6,0.002004,0.004007,0.000000,0.000000,0.01,bool,True,"{'alpha': 0.01, 'fit_prior': 'bool', 'force_al...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
7,0.000804,0.001607,0.000000,0.000000,0.01,bool,bool,"{'alpha': 0.01, 'fit_prior': 'bool', 'force_al...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
8,0.018380,0.005759,0.017077,0.003381,1.0,True,True,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,1
9,0.002004,0.004008,0.000000,0.000000,1.0,True,bool,"{'alpha': 1.0, 'fit_prior': True, 'force_alpha...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
